In [2]:
# データ分析に必要なライブラリを読み込む
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
files = glob.glob("workspace/train/*.csv")
data_list = []
for file in files:
  data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)

/tmp/ipykernel_141914/397933912.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
/tmp/ipykernel_141914/397933912.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


In [4]:
def data_pre(df):

  nonnull_list = []
  for col in df.columns:
    nonnull = df[col].count()
    if nonnull == 0:
      nonnull_list.append(col)
  df = df.drop(nonnull_list, axis=1)

  df["市区町村コード"].value_counts()

  df = df.drop("種類", axis=1)

  distance = {
    "30分?60分":45,
    "1H?1H30":75,
    "2H?":120,
    "1H30?2H":105
  }
  df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(distance).astype(float)

  df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上",2000).astype(float)

  y_list = {}
  for yearStr in df["建築年"].value_counts().keys():
    if "平成" in yearStr:
      num = float(yearStr.split("平成")[1].split("年")[0])
      year = 33 - num
    if "令和" in yearStr:
      num = float(yearStr.split("令和")[1].split("年")[0])
      year = 3 - num
    if "昭和" in yearStr:
      num = float(yearStr.split("昭和")[1].split("年")[0])
      year = 96 - num
      
    y_list[yearStr] = year
  y_list["戦前"] = 76
  df["建築年"] = df["建築年"].replace(y_list)

  year = {
    "年第１四半期":".25",
    "年第２四半期":".50",
    "年第３四半期":".75",
    "年第４四半期":".99"
  }
  yearList = {}
  for yearStr in df["取引時点"].value_counts().keys():
    for k, j in year.items():
      if k in yearStr:
        yearReg = yearStr.replace(k, j)
    
    yearList[yearStr] = yearReg
  df["取引時点"] = df["取引時点"].replace(yearList).astype(float)
  
  # 数値以外のデータ型を"category"と明示
  for col in ["都道府県名", "市区町村名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
    df[col] = df[col].astype("category")
    
  return df

In [5]:
df = data_pre(df)

In [6]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

# モデル学習用のデータ、学習後の検証用データに分割(2割が検証用)
df_train, df_val = train_test_split(df, test_size=0.2)

# 目的変数を設定し目的変数を取り除いた各従属変数を設定する
col = "取引価格（総額）_log"

# 学習用データ
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

# 検証用データ
val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

# light-bgmに渡すパラメータ
params = {
  "objective":"regression",
  "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)

/usr/local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.077781 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12113
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 18
[LightGBM] [Info] Start training from score 7.217129
[1]	valid_0's l1: 0.252339
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.235067


/usr/local/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[3]	valid_0's l1: 0.219814
[4]	valid_0's l1: 0.206202
[5]	valid_0's l1: 0.194026
[6]	valid_0's l1: 0.183563
[7]	valid_0's l1: 0.174401
[8]	valid_0's l1: 0.166167
[9]	valid_0's l1: 0.158898
[10]	valid_0's l1: 0.152421
[11]	valid_0's l1: 0.146704
[12]	valid_0's l1: 0.141492
[13]	valid_0's l1: 0.13666
[14]	valid_0's l1: 0.132522
[15]	valid_0's l1: 0.128875
[16]	valid_0's l1: 0.125499
[17]	valid_0's l1: 0.122427
[18]	valid_0's l1: 0.119641
[19]	valid_0's l1: 0.117231
[20]	valid_0's l1: 0.115055
[21]	valid_0's l1: 0.113161
[22]	valid_0's l1: 0.111282
[23]	valid_0's l1: 0.109422
[24]	valid_0's l1: 0.107696
[25]	valid_0's l1: 0.106082
[26]	valid_0's l1: 0.10458
[27]	valid_0's l1: 0.103391
[28]	valid_0's l1: 0.102232
[29]	valid_0's l1: 0.101131
[30]	valid_0's l1: 0.100049
[31]	valid_0's l1: 0.0991356
[32]	valid_0's l1: 0.0982878
[33]	valid_0's l1: 0.097464
[34]	valid_0's l1: 0.0967276
[35]	valid_0's l1: 0.095967
[36]	valid_0's l1: 0.0952725
[37]	valid_0's l1: 0.0946593
[38]	valid_0's l1: 0.094

In [8]:
# modelに検証用データを渡し予測値を算出する
vals = model.predict(val_x)

# 予測値valsと実際の観測値を比較する
mae(vals, val_y)

0.07575992722199462

In [11]:
df_test = pd.read_csv("workspace/test.csv", index_col=0)

# テストデータを読み込む
df_test = data_pre(df_test)

In [16]:
# モデルに予測用の変数を与え、取引価格の予測結果を取得する
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict

In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19466 entries, 1000000 to 47007421
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   市区町村コード       19466 non-null  int64   
 1   都道府県名         19466 non-null  category
 2   市区町村名         19466 non-null  category
 3   地区名           19463 non-null  category
 4   最寄駅：名称        19453 non-null  category
 5   最寄駅：距離（分）     19453 non-null  float64 
 6   間取り           18544 non-null  category
 7   面積（㎡）         19466 non-null  float64 
 8   建築年           18804 non-null  float64 
 9   建物の構造         18201 non-null  category
 10  用途            13480 non-null  category
 11  今後の利用目的       18439 non-null  category
 12  都市計画          19122 non-null  category
 13  建ぺい率（％）       19045 non-null  float64 
 14  容積率（％）        19045 non-null  float64 
 15  取引時点          19466 non-null  float64 
 16  改装            17036 non-null  category
 17  取引の事情等        152 non-null    category
 1

In [23]:
df_test[["取引価格（総額）_log"]].to_csv("workspace/submit_test.csv")

In [32]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,7816
最寄駅：名称,4875
建築年,3502
取引時点,3305
最寄駅：距離（分）,2228
面積（㎡）,2154
市区町村名,2049
間取り,915
市区町村コード,662
改装,613
